In [96]:
import pyodbc

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=projectdataanalytics.database.windows.net;'
                      'Database=Challenge9b;'
                      'uid=hackadmin;'
                     'pwd=Hack777!')
cursor = conn.cursor()


In [97]:
import pandas as pd

In [98]:
# ReadData

data = pd.read_csv('DataHack7.csv')

In [99]:
data['Baseline Start Date'] = data['Baseline Start Date'].ffill()
data['Baseline Start Date']  = pd.to_datetime(data['Baseline Start Date']).dt.date
data['Baseline End Date'] = data['Baseline End Date'].ffill()
data['Baseline End Date']  = pd.to_datetime(data['Baseline End Date']).dt.date
data['Actual Start Date'] = data['Actual Start Date'].ffill()
data['Actual Start Date']  = pd.to_datetime(data['Actual Start Date']).dt.date
data['Actual End Date'] = data['Actual End Date'].ffill()
data['Actual End Date']  = pd.to_datetime(data['Actual End Date']).dt.date
data['Current Forecast End Date']  = pd.to_datetime(data['Current Forecast End Date']).dt.date
data['Current Forecast Start Date']  = pd.to_datetime(data['Current Forecast Start Date']).dt.date


data.head()

,Milestone/Phase,Milestone Type,PHASE,Resources,Progress (%),Unnamed: 5,Duration,Baseline Start Date,Baseline End Date,Current Forecast Start Date,...,Unnamed: 15,Trend,This Week,Unnamed: 18,Unnamed: 19,Running Total Time PM,Running Total Time BC,Running Total Time DATA,Running Total Time DI,Running Total Time CONFIG
0,Phase,NaN,Discovery,"PM, BC",80,80.0,11.0,2020-08-25,2020-05-09,2020-01-09,...,NaN,0,Green,Green,NaN,#REF!,#REF!,#REF!,#REF!,#REF!
1,Discovery,NaN,Project Plan,PM,80,80.0,NaN,2020-08-25,2020-05-09,2020-01-09,...,3.0,-3,Green,Green,NaN,NaN,NaN,NaN,NaN,NaN
2,Discovery,NaN,External Kick off meeting,PM,100,100.0,NaN,2020-08-25,2020-05-09,2020-02-09,...,3.0,-3,Complete,Complete,NaN,NaN,NaN,NaN,NaN,NaN
3,Discovery,NaN,Internal update to wider teams,PM,90,90.0,NaN,2020-08-25,2020-05-09,2020-02-09,...,3.0,-3,Green,Green,NaN,NaN,NaN,NaN,NaN,NaN
4,Discovery,Sign-off,PID sign off,PM,100,100.0,NaN,2020-08-25,2020-05-09,2020-11-08,...,3.0,-3,Complete,Complete,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
ms_data  = data[['Milestone Type']]
ms_data = ms_data[ms_data['Milestone Type'].notnull()]
ms_data = ms_data['Milestone Type'].unique()

In [97]:
ms_data

array(['Sign-off', 'Live'], dtype=object)

In [35]:
for row in ms_data:
    cursor.execute('''
                INSERT INTO Challenge9b.dbo.MilestoneType (MilestoneType)
                VALUES (?)
                ''',
                row
                )
conn.commit()

In [82]:
phase_data = data[['Milestone/Phase', 'PHASE', 'Progress (%)']]
phase_data = phase_data[phase_data['Milestone/Phase'] == 'Phase']
phase_data = phase_data[phase_data['PHASE'].notna()]
phase_data = phase_data['PHASE'].unique()
phase_data

array(['Discovery', 'Initiation', 'Solution Delivery', 'Deploy', 'Close'],
      dtype=object)

In [69]:
for row in phase_data:
    cursor.execute('''
                INSERT INTO Challenge9b.dbo.Phase (Phase)
                VALUES (?)
                ''',
                row
                )
conn.commit()

In [83]:
RAG_data = data[['This Week']]
RAG_data = RAG_data[RAG_data['This Week'].notna()]
RAG_data = RAG_data['This Week'].unique()
RAG_data

array(['Green', 'Complete', 'Red', 'Amber'], dtype=object)

In [59]:
for row in RAG_data:
    cursor.execute('''
                INSERT INTO Challenge9b.dbo.RAGStatus (RAGStatus)
                VALUES (?)
                ''',
                row
                )
    print(row)
conn.commit()

Green
Complete
Red
Amber


In [5]:
rsrc_data = data['Resources']
rsrc_data = rsrc_data[rsrc_data.notna()]
rsrc_data = rsrc_data.unique()
rsrc_data

array(['PM, BC', 'PM', 'PM, BC, CONFIG, PURCHASE, COMMS, OR, CLIENT, ',
       'BC', 'Conf team', 'Purchase team', 'Comms', 'Ops readiness',
       'Client', 'Ops readiness / PM', 'PURCHASE', 'OM'], dtype=object)

In [6]:
for row in rsrc_data:
    if len(row) <=100:
        cursor.execute('''
                    INSERT INTO Challenge9b.dbo.Resources (ResourceName, ResourceCode)
                    VALUES (?,?)
                    ''',
                    row,
                    row
                    )
conn.commit()

In [100]:
#Tasks
# def get_phase(df):
#     cursor.execute("select * from Phase where Phase = ?", df['Milestone/Phase'])
# #     row = cursor.fetchall() 

tasks = data[['Milestone/Phase', 'Resources', 'This Week','Milestone Type','PHASE', 'Progress (%)','Baseline Start Date', 'Baseline End Date', 'Current Forecast Start Date',
       'Current Forecast End Date', 'Actual Start Date', 'Actual End Date']]
tasks = tasks[tasks['Milestone/Phase']!= 'Phase']
tasks = tasks[tasks['PHASE'].notna()]
# get_phase(tasks)
tasks

,Milestone/Phase,Resources,This Week,Milestone Type,PHASE,Progress (%),Baseline Start Date,Baseline End Date,Current Forecast Start Date,Current Forecast End Date,Actual Start Date,Actual End Date
1,Discovery,PM,Green,NaN,Project Plan,80,2020-08-25,2020-05-09,2020-01-09,2020-09-09,2020-02-09,2020-08-30
2,Discovery,PM,Complete,NaN,External Kick off meeting,100,2020-08-25,2020-05-09,2020-02-09,2020-02-09,2020-02-09,2020-08-30
3,Discovery,PM,Green,NaN,Internal update to wider teams,90,2020-08-25,2020-05-09,2020-02-09,2020-03-16,2020-02-09,2020-08-30
4,Discovery,PM,Complete,Sign-off,PID sign off,100,2020-08-25,2020-05-09,2020-11-08,2020-11-08,2020-02-09,2020-08-30
6,Initiation,PM,Green,NaN,Project Plan,80,2020-05-08,2020-05-09,2020-01-09,2020-09-09,2020-08-13,2020-10-09
7,Initiation,PM,Complete,NaN,External Kick off meeting,100,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09
8,Initiation,PM,Red,NaN,Internal update to wider teams,90,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09
9,Initiation,PM,Amber,NaN,Internal update to wider teams,91,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09
10,Initiation,PM,Complete,Sign-off,PID sign off,100,2020-05-08,2020-05-09,2020-11-08,2020-11-08,2020-08-13,2020-10-09
12,Solution Delivery,BC,Green,NaN,Analysis,75,2020-08-25,2020-10-20,2020-01-09,2020-11-09,2020-01-09,2020-10-28


In [101]:
data.columns

Index(['Milestone/Phase', 'Milestone Type', 'PHASE', 'Resources',
       'Progress (%)', 'Unnamed: 5', 'Duration', 'Baseline Start Date',
       'Baseline End Date', 'Current Forecast Start Date',
       'Current Forecast End Date', 'Actual Start Date', 'Actual End Date',
       'Last Week', 'Unnamed: 14', 'Unnamed: 15', 'Trend', 'This Week',
       'Unnamed: 18', 'Unnamed: 19', 'Running Total Time PM',
       'Running Total Time BC', 'Running Total Time DATA',
       'Running Total Time DI', 'Running Total Time CONFIG'],
      dtype='object')

In [103]:
def get_phase(row):
    cursor.execute("select * from Phase where Phase = ?", row['Milestone/Phase'])
    ret =cursor.fetchone()
    if ret:
        return int(ret[0])
    else:
        None
        
def get_mstype(row):
    if row['Milestone Type']:
        cursor.execute("select * from MilestoneType where MilestoneType = ?", row['Milestone Type'])
        ret =cursor.fetchone()
        if ret:
            return int(ret[0])
    else:
        None

def get_rsrc(row):
    if row['Resources']:
        cursor.execute("select * from Resources where ResourceCode = ?", row['Resources'])
        ret =cursor.fetchone()
        if ret:
            return int(ret[0])
    else:
        None
        
def get_rag(row):
    if row['This Week']:
        cursor.execute("select * from RAGStatus where RAGStatus = ?", row['This Week'])
        ret =cursor.fetchone()
        if ret:
            return int(ret[0])
    else:
        None
tasks['PhaseId'] = tasks.apply(get_phase, axis=1)
tasks['MSTypeId'] = tasks[tasks['Milestone Type'].notna()].apply(get_mstype, axis=1)
tasks['ClientId'] = 1
tasks['ProjectId'] = 1
tasks['Resourceid'] = tasks.apply(get_rsrc, axis=1)
tasks['RAGStatusId'] = tasks.apply(get_rag, axis=1)
tasks

,Milestone/Phase,Resources,This Week,Milestone Type,PHASE,Progress (%),Baseline Start Date,Baseline End Date,Current Forecast Start Date,Current Forecast End Date,Actual Start Date,Actual End Date,PhaseId,MSTypeId,ClientId,ProjectId,Resourceid,RAGStatusId
1,Discovery,PM,Green,NaN,Project Plan,80,2020-08-25,2020-05-09,2020-01-09,2020-09-09,2020-02-09,2020-08-30,54,NaN,1,1,2,1
2,Discovery,PM,Complete,NaN,External Kick off meeting,100,2020-08-25,2020-05-09,2020-02-09,2020-02-09,2020-02-09,2020-08-30,54,NaN,1,1,2,7
3,Discovery,PM,Green,NaN,Internal update to wider teams,90,2020-08-25,2020-05-09,2020-02-09,2020-03-16,2020-02-09,2020-08-30,54,NaN,1,1,2,1
4,Discovery,PM,Complete,Sign-off,PID sign off,100,2020-08-25,2020-05-09,2020-11-08,2020-11-08,2020-02-09,2020-08-30,54,1.0,1,1,2,7
6,Initiation,PM,Green,NaN,Project Plan,80,2020-05-08,2020-05-09,2020-01-09,2020-09-09,2020-08-13,2020-10-09,55,NaN,1,1,2,1
7,Initiation,PM,Complete,NaN,External Kick off meeting,100,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09,55,NaN,1,1,2,7
8,Initiation,PM,Red,NaN,Internal update to wider teams,90,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09,55,NaN,1,1,2,8
9,Initiation,PM,Amber,NaN,Internal update to wider teams,91,2020-05-08,2020-05-09,2020-02-09,2020-02-09,2020-08-13,2020-10-09,55,NaN,1,1,2,9
10,Initiation,PM,Complete,Sign-off,PID sign off,100,2020-05-08,2020-05-09,2020-11-08,2020-11-08,2020-08-13,2020-10-09,55,1.0,1,1,2,7
12,Solution Delivery,BC,Green,NaN,Analysis,75,2020-08-25,2020-10-20,2020-01-09,2020-11-09,2020-01-09,2020-10-28,56,NaN,1,1,4,1


In [104]:
tasks.dtypes

Milestone/Phase                 object
Resources                       object
This Week                       object
Milestone Type                  object
PHASE                           object
Progress (%)                    object
Baseline Start Date             object
Baseline End Date               object
Current Forecast Start Date     object
Current Forecast End Date       object
Actual Start Date               object
Actual End Date                 object
PhaseId                          int64
MSTypeId                       float64
ClientId                         int64
ProjectId                        int64
Resourceid                       int64
RAGStatusId                      int64
dtype: object

In [113]:
for index,row in tasks.iterrows():
    print(row['MSTypeId'])
    isMS = False
    MsID = None
    if row['MSTypeId']>0:
        isMS = True
        MsID = row['MSTypeId']
    cursor.execute('''
                INSERT INTO Challenge9b.dbo.Tasks (
                    Task
                   , Progress
                   , BaselineStart
                   , BaselineEnd
                   , ForecastStart
                   , ForecastEnd
                   , ActualStart
                   , ActualEnd
                   , MilestoneTypeId
                   , isMilestone
                   , ClientId
                   ,ProjectId
                   ,PhaseId
                   ,ResourceId
                   ,RAGStatusId)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                ''',
                row['PHASE'],
                row['Progress (%)'],
                row['Baseline Start Date'],
                row['Baseline End Date'],
                row['Current Forecast Start Date'],
                row['Current Forecast End Date'],
                row['Actual Start Date'],
                row['Actual End Date'],
                MsID,
                isMS,
                row['ClientId'],
                row['ProjectId'],
                row['PhaseId'],
                row['Resourceid'],
                row['RAGStatusId']
                )
conn.commit()

nan
nan
nan
1.0
nan
nan
nan
nan
1.0
nan
nan
nan
nan
nan
nan
nan
2.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
